<a href="https://colab.research.google.com/github/phpons/Neural-Networks-Study/blob/master/NN_Definition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classes Definition

In [ ]:
import numpy as np

class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        self.output = 0

    def activation_function(self, z): # Switch between RELU and Sigmoid as wanted
        return self.sigmoid(z)

    def relu(self, z):
        return max(0, z)

    def sigmoid(self, z):
        if abs(z) > 10:
          # Avoids overflow
          out = 1 if z > 0 else 0
        else:
          out = 1.0 / (1.0 + np.exp(-z))
        return out

    def de_dz(self, de_dy):
        return (self.output * (1- self.output)) * de_dy

    def de_dz_relu(self, de_dy):
        if self.output > 0:
            return 1
        else:
            return 0

    def z(self, inputs):
        sum_input = 0
        for i, weight in enumerate(self.weights):
            sum_input += weight * inputs[i]
        return sum_input + self.bias

    def y(self, inputs):
        self.output = self.activation_function(self.z(inputs))
        return self.output


class DenseLayer:

    def __init__(self, neurons):
        self.neurons = neurons
        self.outputs = []
        self.outputs = np.array(self.outputs)

    def feed_forward(self, inputs):
        outputs = []

        for i, neuron in enumerate(self.neurons):
            outputs.append(neuron.y(inputs))

        outputs = np.array(outputs)
        return outputs


class NeuralNetwork:

    def __init__(self, sizes):
        self.num_layers = len(sizes) - 1
        self.layers = []

        # Generates weights and biases
        weights_all = [np.random.randn(y, x)
                   for x, y in zip(sizes[:-1], sizes[1:])]
        weights_all[:] = [x / 12 for x in weights_all]

        biases = [np.random.randn(y, 1) for y in sizes[1:]]

        for i in range(self.num_layers): # Creates layers
            neurons = []

            for j in range(sizes[i+1]):
                weights = np.random.uniform(-1, 1, sizes[i])
                neuron = Neuron(weights_all[i][j], biases[i][j][0])
                neurons.append(neuron)

            self.layers.append(DenseLayer(neurons))

    def feed_forward(self, inputs):
        pass_forward = inputs
        for i, layer in enumerate(self.layers):
            pass_forward = self.layers[i].feed_forward(pass_forward)
        return pass_forward

    def cost_function(self, output, expected_output):
        output_array = np.array(output)
        expected_output_array = np.array(expected_output)

        sq_error = (expected_output_array - output_array) ** 2
        mean_sq_error = np.sum(sq_error) / (sq_error.size*2)

        return mean_sq_error

    def de_dy(self, expected_output, output): #DE/DYj
        return -1 * (expected_output - output)

    def backprop(self, expected_output, output, inputs, learning_rate):
        de_dy_array = self.de_dy(expected_output, output)

        for i in reversed(range(len(self.layers))):
            if i != 0:
                for j, neuron in enumerate(self.layers[i].neurons):
                    if i == (len(self.layers) - 1): # Last layer
                        de_dz = neuron.de_dz(de_dy_array[j])
                    else:
                        de_dz = neuron.de_dz_relu(de_dy_array[j])
                    de_dyi = np.zeros(len(neuron.weights)) # Derivative from previous layer

                    for idx, w in enumerate(neuron.weights):
                        de_dw = de_dz * self.layers[i - 1].neurons[idx].output # de_dz * Yi
                        de_dyi[idx] += de_dz * w
                        self.layers[i].neurons[j].weights[idx] -= learning_rate * de_dw # Updates weights

                    self.layers[i].neurons[j].bias -= learning_rate * de_dz # Updates biases
                de_dy_array = np.copy(de_dyi)

            else: #já que input não está em self.layers, preciso deste else pra pegar os inputs
                for j, neuron in enumerate(self.layers[i].neurons):
                    de_dz = neuron.de_dz(de_dy_array[j])
                    de_dyi = np.zeros(len(neuron.weights))  # Derivative from previous layer

                    for idx, w in enumerate(neuron.weights):
                        de_dw = de_dz * inputs[idx]  # de_dz * Yi
                        self.layers[i].neurons[j].weights[idx] -= learning_rate * de_dw  # Updates weights

                    self.layers[i].neurons[j].bias -= learning_rate * de_dz  # Updates biases
                de_dy_array = np.copy(de_dyi)